In [22]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.metrics import accuracy_score
from ucimlrepo import fetch_ucirepo
from typing import List, Dict
from scipy.special import gammaln

class Node:
    def __init__(self, feature_index=None, threshold=None, left=None, right=None, value=None):
        self.feature_index = feature_index
        self.threshold = threshold
        self.left = left
        self.right = right
        self.value = value

class SMCDecisionTree:
    def __init__(self, max_depth=5, min_samples_split=2, n_particles=100, alpha_value=1.0):
        self.max_depth = max_depth
        self.min_samples_split = min_samples_split
        self.n_particles = n_particles
        self.tree = None
        self.alpha_value = alpha_value

    def _log_dirichlet(self, dirichlet_params):
        return np.sum(gammaln(dirichlet_params)) - gammaln(np.sum(dirichlet_params)) 

    def _initialize_particles(self, n_features):
        return {
            'feature_index': np.random.randint(0, n_features, self.n_particles),
            'threshold': np.random.uniform(0, 1, self.n_particles)
        }

    def _move_particles(self, particles, n_features):
        move_mask = np.random.random(self.n_particles) < 0.5
        particles['feature_index'][move_mask] = np.random.randint(0, n_features, np.sum(move_mask))
        particles['threshold'][~move_mask] += np.random.normal(0, 0.1, np.sum(~move_mask))
        return particles

    def _evaluate_split(self, X, y, feature_index, threshold):
        left_mask = X[:, feature_index] <= threshold
        left_y, right_y = y[left_mask], y[~left_mask]
        
        if len(left_y) == 0 or len(right_y) == 0:
            return -np.inf
        
        left_counts = np.zeros(len(self.classes_))
        right_counts = np.zeros(len(self.classes_))
        
        for i, c in enumerate(self.classes_):
            left_counts[i] = np.sum(left_y == c)
            right_counts[i] = np.sum(right_y == c)
        
        alpha = np.ones(len(self.classes_)) * self.alpha_value
        
        log_likelihood = (
            self._log_dirichlet(left_counts + alpha) - self._log_dirichlet(alpha) +
            self._log_dirichlet(right_counts + alpha) - self._log_dirichlet(alpha)
        )
        
        log_prior = -(np.log2(4) + np.log2(X.shape[1]))*self.num_nodes()
         
        return log_likelihood + log_prior

    def _smc_split(self, X, y):
        particles = self._initialize_particles(X.shape[1])
        
        for _ in range(5):  # Number of SMC iterations
            particles = self._move_particles(particles, X.shape[1])
            
            weights = np.array([self._evaluate_split(X, y, p_f, p_t) 
                                for p_f, p_t in zip(particles['feature_index'], particles['threshold'])])
            weights = np.where(np.isfinite(weights), np.exp(weights), 0)
            
            # Handle potential zero sum of weights
            if np.sum(weights) == 0:
                weights = np.ones_like(weights) / len(weights)
            else:
                weights /= np.sum(weights)
            
            # Check for NaN values and replace with uniform probabilities if necessary
            if np.any(np.isnan(weights)):
                print("Warning: NaN weights encountered. Using uniform probabilities.")
                weights = np.ones_like(weights) / len(weights)
            
            indices = np.random.choice(self.n_particles, size=self.n_particles, p=weights)
            particles = {k: v[indices] for k, v in particles.items()}
        
        best_index = np.argmax(weights)
        return particles['feature_index'][best_index], particles['threshold'][best_index]

    def _grow_tree(self, X, y, depth=0):
        n_samples, n_features = X.shape
        unique_classes = np.unique(y)

        if depth >= self.max_depth or n_samples < self.min_samples_split or len(unique_classes) == 1:
            counts = np.zeros(len(self.classes_))
            for i, c in enumerate(self.classes_):
                counts[i] = np.sum(y == c)
            return Node(value=counts)

        feature_index, threshold = self._smc_split(X, y)
        
        left_mask = X[:, feature_index] <= threshold
        X_left, y_left = X[left_mask], y[left_mask]
        X_right, y_right = X[~left_mask], y[~left_mask]
        
        left_subtree = self._grow_tree(X_left, y_left, depth + 1)
        right_subtree = self._grow_tree(X_right, y_right, depth + 1)

        return Node(feature_index=feature_index, threshold=threshold, left=left_subtree, right=right_subtree)

    def fit(self, X, y):
        self.n_features = X.shape[1]
        self.classes_ = np.unique(y)
        self.n_classes = len(self.classes_)
        print(f"Number of classes: {self.n_classes}")
        print(f"Unique classes: {self.classes_}")
        print(f"Input y shape: {y.shape}")
        print(f"Input y unique values: {np.unique(y)}")
        self.tree = self._grow_tree(X, y)

    def _predict_sample(self, x, node):
        if node.value is not None:
            return self.classes_[np.argmax(node.value)]
        
        if x[node.feature_index] <= node.threshold:
            return self._predict_sample(x, node.left)
        else:
            return self._predict_sample(x, node.right)

    def predict(self, X):
        return np.array([self._predict_sample(x, self.tree) for x in X])

    def num_nodes(self):
        return self._count_nodes(self.tree)

    def _count_nodes(self, node):
        if node is None:
            return 0
        return 1 + self._count_nodes(node.left) + self._count_nodes(node.right)

# ... [rest of the code remains the same] ...


def run_smc_dt_on_dataset(X, y, seed=42):
    try:
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=seed)

        scaler = StandardScaler()
        X_train_scaled = scaler.fit_transform(X_train)
        X_test_scaled = scaler.transform(X_test)

        smc_dt = SMCDecisionTree(max_depth=5, n_particles=100)
        smc_dt.fit(X_train_scaled, y_train)

        y_pred = smc_dt.predict(X_test_scaled)
        accuracy = accuracy_score(y_test, y_pred)

        return accuracy, smc_dt.num_nodes()
    except Exception as e:
        print(f"Error occurred: {str(e)}")
        import traceback
        traceback.print_exc()
        return None, None

def experiment_on_datasets(seeds: List[int], n_trials=10) -> Dict[str, Dict[str, float]]:
    datasets = [
        # (267, "Banknote"),
        (17, "BCW-D"),
        (109, "Wine"),
        (53, "Iris"),
        (850, "Raisin"),
        # (15, "BCW")
    ]

    results = {}

    for dataset_id, dataset_name in datasets:
        print(f"\nRunning experiment on {dataset_name} dataset...")
        dataset = fetch_ucirepo(id=dataset_id)
        X = dataset.data.features.values
        y = dataset.data.targets.values.ravel()

        print(f"Original y shape: {y.shape}")
        print(f"Original y unique values: {np.unique(y)}")

        if y.dtype == object:
            le = LabelEncoder()
            y = le.fit_transform(y)
            print(f"After LabelEncoder - y unique values: {np.unique(y)}")
        
        print(f"Dataset shape: {X.shape}")
        print(f"Unique classes in dataset: {np.unique(y)}")
        
        accuracies = []
        n_nodes_list = []
        for seed in seeds:
            print(f"\nRunning with seed {seed}")
            try:
                accs, sizes = [], []
                for _ in range(n_trials): 
                    accuracy, n_nodes = run_smc_dt_on_dataset(X, y, seed)
                    if accuracy is not None and n_nodes is not None:
                        accs.append(accuracy)
                        sizes.append(n_nodes)
                    accuracies.append(np.mean(accs))
                    n_nodes_list.append(np.mean(sizes))
                    print(f"Seed {seed} - Accuracy: {np.mean(accs):.4f}, Nodes: {np.mean(sizes)}")
            except Exception as e:
                print(f"Error occurred with seed {seed}: {str(e)}")
        
        if accuracies and n_nodes_list:
            mean_accuracy = np.mean(accuracies)
            std_accuracy = np.std(accuracies)
            mean_nodes = np.mean(n_nodes_list)
            std_nodes = np.std(n_nodes_list)
            
            results[dataset_name] = {
                "mean_accuracy": mean_accuracy,
                "std_accuracy": std_accuracy,
                "mean_nodes": mean_nodes,
                "std_nodes": std_nodes
            }
            
            print(f"{dataset_name} - Mean Accuracy: {mean_accuracy:.4f}, Std: {std_accuracy:.4f}")
            print(f"{dataset_name} - Mean Nodes: {mean_nodes:.2f}, Std: {std_nodes:.2f}")
        else:
            print(f"Failed to process {dataset_name} dataset")
    
    return results

# Run the experiments
# np.random.seed(42)
seeds = np.array([1, 2, 3, 4, 5])  # Using multiple seeds for more robust results
results = experiment_on_datasets(seeds, n_trials=10)

# Print final results
print("\nFinal Results:")
for dataset, metrics in results.items():
    print(f"{dataset}:")
    print(f"  Accuracy - Mean: {metrics['mean_accuracy']:.4f}, Std: {metrics['std_accuracy']:.4f}")
    print(f"  Nodes    - Mean: {metrics['mean_nodes']:.2f}, Std: {metrics['std_nodes']:.2f}")


Running experiment on BCW-D dataset...
Original y shape: (569,)
Original y unique values: ['B' 'M']
After LabelEncoder - y unique values: [0 1]
Dataset shape: (569, 30)
Unique classes in dataset: [0 1]

Running with seed 1
Number of classes: 2
Unique classes: [0 1]
Input y shape: (455,)
Input y unique values: [0 1]
Seed 1 - Accuracy: 0.9211, Nodes: 35.0
Number of classes: 2
Unique classes: [0 1]
Input y shape: (455,)
Input y unique values: [0 1]
Seed 1 - Accuracy: 0.9167, Nodes: 37.0
Number of classes: 2
Unique classes: [0 1]
Input y shape: (455,)
Input y unique values: [0 1]
Seed 1 - Accuracy: 0.9298, Nodes: 36.333333333333336
Number of classes: 2
Unique classes: [0 1]
Input y shape: (455,)
Input y unique values: [0 1]
Seed 1 - Accuracy: 0.9276, Nodes: 36.5
Number of classes: 2
Unique classes: [0 1]
Input y shape: (455,)
Input y unique values: [0 1]
Seed 1 - Accuracy: 0.9281, Nodes: 36.2
Number of classes: 2
Unique classes: [0 1]
Input y shape: (455,)
Input y unique values: [0 1]
See

## **MCMC** ##

In [1]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.metrics import accuracy_score
from ucimlrepo import fetch_ucirepo
from typing import List, Dict
from scipy.special import gammaln

class Node:
    def __init__(self, feature_index=None, threshold=None, left=None, right=None, value=None):
        self.feature_index = feature_index
        self.threshold = threshold
        self.left = left
        self.right = right
        self.value = value

class MCMCDecisionTree:
    def __init__(self, max_depth=5, min_samples_split=2, mcmc_iterations=1000, alpha_value=0.1):
        self.max_depth = max_depth
        self.min_samples_split = min_samples_split
        self.mcmc_iterations = mcmc_iterations
        self.tree = None
        self.alpha_value = alpha_value

    def _log_dirichlet(self, dirichlet_params):
        return np.sum(gammaln(dirichlet_params)) - gammaln(np.sum(dirichlet_params))

    def _evaluate_split(self, X, y, feature_index, threshold):
        left_mask = X[:, feature_index] <= threshold
        left_y, right_y = y[left_mask], y[~left_mask]
        
        if len(left_y) == 0 or len(right_y) == 0:
            return -np.inf
        
        left_counts = np.zeros(len(self.classes_))
        right_counts = np.zeros(len(self.classes_))
        
        for i, c in enumerate(self.classes_):
            left_counts[i] = np.sum(left_y == c)
            right_counts[i] = np.sum(right_y == c)
        
        alpha = np.ones(len(self.classes_)) * self.alpha_value
        
        log_likelihood = (
            self._log_dirichlet(left_counts + alpha) - self._log_dirichlet(alpha) +
            self._log_dirichlet(right_counts + alpha) - self._log_dirichlet(alpha)
        )

        log_prior = -(np.log2(4) + np.log2(X.shape[1]))*self.num_nodes()
         
        return log_likelihood + log_prior

    def _mcmc_split(self, X, y):
        n_features = X.shape[1]
        current_feature = np.random.randint(0, n_features)
        current_threshold = np.random.choice(X[:, current_feature])
        current_score = self._evaluate_split(X, y, current_feature, current_threshold)
        
        best_feature, best_threshold, best_score = current_feature, current_threshold, current_score
        
        for _ in range(self.mcmc_iterations):
            if np.random.random() < 0.5:
                proposed_feature = np.random.randint(0, n_features)
                proposed_threshold = np.random.choice(X[:, proposed_feature])
            else:
                proposed_feature = current_feature
                proposed_threshold = current_threshold + np.random.normal(0, 0.1 * (np.max(X[:, current_feature]) - np.min(X[:, current_feature])))
            
            proposed_score = self._evaluate_split(X, y, proposed_feature, proposed_threshold)
            
            if np.isfinite(proposed_score) and np.isfinite(current_score):
                acceptance_prob = min(1, np.exp(proposed_score - current_score))
            elif np.isfinite(proposed_score):
                acceptance_prob = 1
            else:
                acceptance_prob = 0
            
            if np.random.random() < acceptance_prob:
                current_feature, current_threshold, current_score = proposed_feature, proposed_threshold, proposed_score
                
                if current_score > best_score:
                    best_feature, best_threshold, best_score = current_feature, current_threshold, current_score
        
        return best_feature, best_threshold

    def _grow_tree(self, X, y, depth=0):
        n_samples, n_features = X.shape
        unique_classes = np.unique(y)

        if depth >= self.max_depth or n_samples < self.min_samples_split or len(unique_classes) == 1:
            counts = np.zeros(len(self.classes_))
            for i, c in enumerate(self.classes_):
                counts[i] = np.sum(y == c)
            return Node(value=counts)

        feature_index, threshold = self._mcmc_split(X, y)
        
        left_mask = X[:, feature_index] <= threshold
        X_left, y_left = X[left_mask], y[left_mask]
        X_right, y_right = X[~left_mask], y[~left_mask]
        
        left_subtree = self._grow_tree(X_left, y_left, depth + 1)
        right_subtree = self._grow_tree(X_right, y_right, depth + 1)

        return Node(feature_index=feature_index, threshold=threshold, left=left_subtree, right=right_subtree)

    def fit(self, X, y):
        self.n_features = X.shape[1]
        self.classes_ = np.unique(y)
        self.n_classes = len(self.classes_)
        print(f"Number of classes: {self.n_classes}")
        print(f"Unique classes: {self.classes_}")
        print(f"Input y shape: {y.shape}")
        print(f"Input y unique values: {np.unique(y)}")
        self.tree = self._grow_tree(X, y)

    def _predict_sample(self, x, node):
        if node.value is not None:
            return self.classes_[np.argmax(node.value)]
        
        if x[node.feature_index] <= node.threshold:
            return self._predict_sample(x, node.left)
        else:
            return self._predict_sample(x, node.right)

    def predict(self, X):
        return np.array([self._predict_sample(x, self.tree) for x in X])

    def num_nodes(self):
        return self._count_nodes(self.tree)

    def _count_nodes(self, node):
        if node is None:
            return 0
        return 1 + self._count_nodes(node.left) + self._count_nodes(node.right)

def run_mcmc_dt_on_dataset(X, y, seed=42):
    try:
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=seed)

        scaler = StandardScaler()
        X_train_scaled = scaler.fit_transform(X_train)
        X_test_scaled = scaler.transform(X_test)

        mcmc_dt = MCMCDecisionTree(max_depth=5)
        mcmc_dt.fit(X_train_scaled, y_train)

        y_pred = mcmc_dt.predict(X_test_scaled)
        accuracy = accuracy_score(y_test, y_pred)

        return accuracy, mcmc_dt.num_nodes()
    except Exception as e:
        print(f"Error occurred: {str(e)}")
        import traceback
        traceback.print_exc()
        return None, None
    
# ... [rest of the code remains the same] ...
def experiment_on_datasets(seeds: List[int], n_trials: int=1) -> Dict[str, Dict[str, float]]:
    datasets = [
        # (267, "Banknote"),
        (17, "BCW-D"),
        (109, "Wine"),
        (53, "Iris"),
        (850, "Raisin"),
        # (15, "BCW")
    ]

    results = {}

    for dataset_id, dataset_name in datasets:
        print(f"\nRunning experiment on {dataset_name} dataset...")
        dataset = fetch_ucirepo(id=dataset_id)
        X = dataset.data.features.values
        y = dataset.data.targets.values.ravel()

        print(f"Original y shape: {y.shape}")
        print(f"Original y unique values: {np.unique(y)}")

        if y.dtype == object:
            le = LabelEncoder()
            y = le.fit_transform(y)
            print(f"After LabelEncoder - y unique values: {np.unique(y)}")
        
        print(f"Dataset shape: {X.shape}")
        print(f"Unique classes in dataset: {np.unique(y)}")
        
        accuracies = []
        n_nodes_list = []
        for seed in seeds:
            print(f"\nRunning with seed {seed}")
            accs, sizes = [], []
            for _ in range(n_trials): 
                accuracy, n_nodes = run_mcmc_dt_on_dataset(X, y, seed)
                if accuracy is not None and n_nodes is not None:
                    accs.append(accuracy)
                    sizes.append(n_nodes)
                accuracies.append(np.mean(accs))
                n_nodes_list.append(np.mean(sizes))
                print(f"Seed {seed} - Accuracy: {np.mean(accs):.4f}, Nodes: {np.mean(sizes)}")
            if accuracy is not None and n_nodes is not None:
                accuracies.append(accuracy)
                n_nodes_list.append(n_nodes)
                print(f"Seed {seed} - Accuracy: {accuracy:.4f}, Nodes: {n_nodes}")
        
        if accuracies and n_nodes_list:
            mean_accuracy = np.mean(accuracies)
            std_accuracy = np.std(accuracies)
            mean_nodes = np.mean(n_nodes_list)
            std_nodes = np.std(n_nodes_list)
            
            results[dataset_name] = {
                "mean_accuracy": mean_accuracy,
                "std_accuracy": std_accuracy,
                "mean_nodes": mean_nodes,
                "std_nodes": std_nodes
            }
            
            print(f"{dataset_name} - Mean Accuracy: {mean_accuracy:.4f}, Std: {std_accuracy:.4f}")
            print(f"{dataset_name} - Mean Nodes: {mean_nodes:.2f}, Std: {std_nodes:.2f}")
        else:
            print(f"Failed to process {dataset_name} dataset")
    
    return results

# Run the experiments
np.random.seed(42)
seeds = np.array([1, 2, 3, 4, 5])  # Using multiple seeds for more robust results
results = experiment_on_datasets(seeds, n_trials=10)

# Print final results
print("\nFinal Results:")
for dataset, metrics in results.items():
    print(f"{dataset}:")
    print(f"  Accuracy - Mean: {metrics['mean_accuracy']:.4f}, Std: {metrics['std_accuracy']:.4f}")
    print(f"  Nodes    - Mean: {metrics['mean_nodes']:.2f}, Std: {metrics['std_nodes']:.2f}")


Running experiment on BCW-D dataset...
Original y shape: (569,)
Original y unique values: ['B' 'M']
After LabelEncoder - y unique values: [0 1]
Dataset shape: (569, 30)
Unique classes in dataset: [0 1]

Running with seed 1
Number of classes: 2
Unique classes: [0 1]
Input y shape: (455,)
Input y unique values: [0 1]
Seed 1 - Accuracy: 0.9386, Nodes: 29.0
Number of classes: 2
Unique classes: [0 1]
Input y shape: (455,)
Input y unique values: [0 1]
Seed 1 - Accuracy: 0.9254, Nodes: 29.0
Number of classes: 2
Unique classes: [0 1]
Input y shape: (455,)
Input y unique values: [0 1]
Seed 1 - Accuracy: 0.9298, Nodes: 29.0
Number of classes: 2
Unique classes: [0 1]
Input y shape: (455,)
Input y unique values: [0 1]
Seed 1 - Accuracy: 0.9320, Nodes: 27.5
Number of classes: 2
Unique classes: [0 1]
Input y shape: (455,)
Input y unique values: [0 1]
Seed 1 - Accuracy: 0.9368, Nodes: 27.4
Number of classes: 2
Unique classes: [0 1]
Input y shape: (455,)
Input y unique values: [0 1]
Seed 1 - Accuracy